# Database Split Rationale

## PostgreSQL for:

### 1. **User Data & Authentication**

1. Structured data with fixed schema
2. ACID compliance for critical user information
3. Strong referential integrity for user relationships
4. Secure transactions for authentication operations



### 2. **Logging & Tracking Data**

1. Time-series data with consistent structure
2. Relational queries for user activity
3. Transaction support for data integrity
4. Complex joins between related activities



### 3. **Goals & Progress**

1. Structured data with clear relationships
2. Consistent schema for tracking progress
3. Transaction support for updates
4. Foreign key constraints for data integrity

## MongoDB for:

## 1. **Food Database**

1. Varying attributes across different food types
2. Flexible schema for different nutrition profiles
3. Large dataset with complex nested structures
4. Text search capabilities for food lookup
5. Easy updates to food database without migrations



## 2. **Recipes & Custom Foods**

1. Varying ingredients and preparation steps
2. Nested arrays and objects for ingredients
3. Flexible schema for different recipe types
4. Rich text and media content



## 3. **Aggregated Analytics**

1. Denormalized data for reporting
2. Flexible schema for different analysis types
3. Document-oriented structure for complex metrics
4. Efficient storage of pre-calculated statistics

## Performance Optimization Strategies

### PostgreSQL Optimization

## 1. **Partitioning**

1. Partition time-series tables (weight_logs, meal_logs, workout_logs) by date ranges
2. Example: Partition by month for recent data, by year for older data

In [ ]:
# SQL file

CREATE TABLE weight_logs (
    id UUID NOT NULL,
    user_id UUID NOT NULL,
    weight_kg NUMERIC(5,2) NOT NULL,
    log_date DATE NOT NULL,
    notes TEXT,
    created_at TIMESTAMP WITH TIME ZONE DEFAULT NOW()
) PARTITION BY RANGE (log_date);

CREATE TABLE weight_logs_current_month PARTITION OF weight_logs
    FOR VALUES FROM ('2023-05-01') TO ('2023-06-01');

CREATE TABLE weight_logs_previous_months PARTITION OF weight_logs
    FOR VALUES FROM ('2022-01-01') TO ('2023-05-01');

## 2. **Materialized Views**

- Create materialized views for common analytics queries
- Refresh on schedule or after significant data changes

In [ ]:
# SQL file

CREATE TABLE weight_logs (
    id UUID NOT NULL,
    user_id UUID NOT NULL,
    weight_kg NUMERIC(5,2) NOT NULL,
    log_date DATE NOT NULL,
    notes TEXT,
    created_at TIMESTAMP WITH TIME ZONE DEFAULT NOW()
) PARTITION BY RANGE (log_date);

CREATE TABLE weight_logs_current_month PARTITION OF weight_logs
    FOR VALUES FROM ('2023-05-01') TO ('2023-06-01');

CREATE TABLE weight_logs_previous_months PARTITION OF weight_logs
    FOR VALUES FROM ('2022-01-01') TO ('2023-05-01');

## 3.  **Composite Indexes**

- Create composite indexes for common query patterns

In [ ]:
# SQL File

-- For queries that filter by user and date range
CREATE INDEX idx_meal_logs_user_date_meal_type ON meal_logs(user_id, meal_time, meal_type);

-- For queries that join workout_exercises with exercises
CREATE INDEX idx_workout_exercises_workout_exercise ON workout_exercises(workout_log_id, exercise_id);

# MongoDB Optimization

##  1. **Compound Indexes**

1. Create compound indexes for common query patterns

In [ ]:
# JS File

// For queries that search foods by name within categories
db.foods.createIndex({ "categories": 1, "name": 1 });

// For queries that filter recipes by author and categories
db.recipes.createIndex({ "author_id": 1, "categories": 1 });

## 2. **Covered Queries**

- Design indexes to cover common queries

In [ ]:
# JS File

// For queries that only need name and nutrition info
db.foods.createIndex({ 
    "name": 1, 
    "nutrition_per_100g.calories": 1,
    "nutrition_per_100g.protein_g": 1,
    "nutrition_per_100g.carbs_g": 1,
    "nutrition_per_100g.fat_g": 1
});

## 3. **Aggregation Pipeline Optimization**

- Use indexes in aggregation stages
- Limit document size in early stages

In [ ]:
# JS FIle

// Example of optimized aggregation for user nutrition trends
db.nutrition_logs.aggregate([
    { $match: { user_id: "user123" } },  // Uses index on user_id
    { $sort: { date: -1 } },             // Uses index on date
    { $limit: 30 },                      // Limits documents early
    { $project: {                        // Projects only needed fields
        date: 1,
        "daily_totals.calories": 1,
        "daily_totals.protein_g": 1,
        "daily_totals.carbs_g": 1,
        "daily_totals.fat_g": 1
    }}
]);

# Data Synchronization Strategy

## To maintain consistency between PostgreSQL and MongoDB:

## 1. **Unique Identifiers**

1. Use UUIDs as primary keys in PostgreSQL
2. Reference these UUIDs in MongoDB documents
3. Example: meal_items in PostgreSQL references food_id in MongoDB



## 2. **Application-Level Transactions**

1. Implement two-phase commit pattern for operations spanning both databases
2. Use compensating transactions for rollback



## 3. **Change Data Capture**

1. Use PostgreSQL logical replication to capture changes
2. Process changes to update corresponding MongoDB documents
3. Example: When a meal_log is created in PostgreSQL, update nutrition_logs in MongoDB



## 4. **Periodic Reconciliation**

1. Run scheduled jobs to verify data consistency
2. Repair inconsistencies automatically or flag for review





### Backup and Recovery Strategy

## 1. **PostgreSQL Backup**

1. Daily full backups using pg_dump
2. Continuous WAL archiving for point-in-time recovery
3. Retention policy: 30 days of daily backups, 12 months of monthly backups



## 2. **MongoDB Backup**

1. Daily full backups using mongodump
2. Continuous oplog tailing for point-in-time recovery
3. Retention policy: 30 days of daily backups, 12 months of monthly backups



## 3. **Cross-Database Consistency**

1. Coordinate backups between PostgreSQL and MongoDB
2. Document reference points between databases
3. Test recovery procedures regularly